In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import copy
import glob
import random
from random import random as rd

# files are from 
* https://www.kaggle.com/alexryzhkov/sep21-lightautoml-starter by Alexander Ryzhkov
* https://www.kaggle.com/alexryzhkov/sep21-autowoe-interpretable-model-starter by Alexander Ryzhkov
* https://www.kaggle.com/tomoyaogawam/tps-sep-lightgbm-baseline-2 by TOMH
* https://www.kaggle.com/kennethquisado/xgboost-10fold-cv-baseline by Kenneth Q
* https://www.kaggle.com/maksymshkliarevskyi/tps-sep-starting-point-eda-xgb-cv-baseline by Maksym Shkliarevskyi
* https://www.kaggle.com/pratikkgandhi/histgradientboostingclf-baseline by Pratik Gandhi
* https://www.kaggle.com/kieranmcgee/tps-september-lightautoml-baseline/output?scriptVersionId=73682974 by Kieran McGee
* https://www.kaggle.com/lukaszborecki/tps-09-nn by Lukasz Borecki
* https://www.kaggle.com/suharkov/sep-2021-playground-from-russia-with-love/output by Suharkov
* https://www.kaggle.com/antonellomartiello/tps09-autogluon/output?scriptVersionId=73716407&select=submission.csv by Antonello Martiello
* https://www.kaggle.com/lucamassaron/autogluon-for-tabular-playground-sep-2021 by Luca Massaron
* https://www.kaggle.com/farelarden/tabular-sept-2021-optimized-xgb by Farel Arden
* https://www.kaggle.com/rahullalu/tps-sept-2021-eda-and-base-models/output?scriptVersionId=73706363 by rahul yadav
* https://www.kaggle.com/azzamradman/catboost-baseline-with-simple-eda/output?scriptVersionId=73682055&select=catboost_test.csv by Azzam Radman
* https://www.kaggle.com/mrigendraagrawal/tps-sep-eda-and-starter by Mrigendra Agrawal
* https://www.kaggle.com/venkatkumar001/tps-sep-21-day-1-baseline-xgb-fold/output?scriptVersionId=73725776 by Venkatkumar R
* https://www.kaggle.com/hamzaghanmi/tps-sep-starter-eda-xgboost by Hamza
* https://www.kaggle.com/junhyeok99/tps-xgb-lgbm by JunHyeok Park
* https://www.kaggle.com/junhyeok99/stacking-ensemble-for-beginner by JunHyeok Park
* https://www.kaggle.com/hiro5299834/tps-sep-2021-single-lgbm by BIZEN
* https://www.kaggle.com/mlanhenke/tps-09-single-catboostclassifier-0-81676 by mlanhenke
* https://www.kaggle.com/aleksandraposka/mljar-automl-starter-optuna-mode-tps-sep-21 by Aleksandra Płońska
* https://www.kaggle.com/tensorchoko/tabular-sep-2021-lightgbm/output by tensor choko
* https://www.kaggle.com/takahiroyoshida012/eda-lightgbm-hyperparameter-tuning-using-optuna by Momota
* https://www.kaggle.com/manabendrarout/classification-on-gpu-lgb-and-xgboost by Manav
* https://www.kaggle.com/hsuchialun/tps-xgboost-kfold-with-gpu/output?scriptVersionId=73870353 by Hsu,CHIA-LUN
* https://www.kaggle.com/maximkazantsev/tps-09-21-eda-xgboost-with-folds/output by Maxim Kazantsev
* https://www.kaggle.com/nitinrajput47/tps-aug-light-gbm/output by Nitin Rajput
* https://www.kaggle.com/maximkazantsev/tps-09-21-eda-lightgbm-with-folds/data?select=submission.csv by Maxim Kazantsev
* https://www.kaggle.com/vishwas21/tps-sep-21-tuning-xgb-catb-lgbm-and-stacking/output?select=submission.csv by Vishwas Chepuri
* https://www.kaggle.com/hsuchialun/tps-lightgbm-kfold/output by Hsu,CHIA-LUN
* https://www.kaggle.com/ranjeetshrivastav/catboost-lightgbm/output by Ranjeet shrivastav
* https://www.kaggle.com/munumbutt/automl-autogluon/ by Munum Butt

In [ ]:
def set_seed(seed = 2021):
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state

def wc(input_list):
    # weight calculation based on public lb score

    # the weight W1 of file 1 with pulibc lb score L1 equals
    # W1 = L1 / (L1 + L2 + ...)
    
    l = []
    for i in range(len(input_list)):
        public_lb_score = int(input_list[i].split("/")[-1].split(".")[1].split(" ")[0]) 
        l.append(public_lb_score)
        input_list[i] = (pd.read_csv(input_list[i]).claim).ravel()
    output = 0
    l_sum = sum(l)
    for i in range(len(input_list)):
        weight = l[i] / l_sum
        output += input_list[i] * weight
    return output

def generate(dp):
    # get all the files to blend
    l = []
    for i in glob.iglob(f'{dp}/*'):
        l.append(i)
    file_count = len(l)
    loop_time = file_count ** 3
    
    # calculate the number of files in the input dataset
    # and split them in pairs
    splits = file_count // 2
    
    # sort the file based on their public lb score
    l.sort(reverse = True)
    flist = []
    
    # create a file list
    # append the 2 by 2 files as one element
    # in the last loop, append all the files which are not necessarily 2 files
    for i in range(splits):
        if i == splits - 1:
            flist.append(l[i * round(file_count / splits): ])
        else:
            flist.append(l[i * round(file_count / splits): (i + 1) * round(file_count / splits)])
    
    # transfrom each element in the file list into one blended prediction
    for i in range(len(flist)):   
        flist[i] = wc(flist[i])
    output = pd.read_csv("../input/tabular-playground-series-sep-2021/sample_solution.csv")
    output.claim = 0       
    
    # loop a large number of times
    # to converge the result into a stable expected value
    for i in range(loop_time):      
        weight = []        
        set_seed(i)
        
        # create a weight list with the same length as the file list
        for i in range(len(flist)):
            weight.append(rd())
        weight_sum = sum(weight)
        
        # normalize the weights
        for i in range(len(weight)):
            weight[i] /= weight_sum
        weight.sort(reverse = True)
        
        # assign each weights to each blended prediction
        for i in range(len(flist)):
            output.claim += flist[i] * weight[i]    
    output.claim /= (loop_time)
    output.to_csv(f'rwb {loop_time} loops version 24.csv',index=False)

In [ ]:
generate('../input/tps09-public-notebook')